In [1]:
import numpy as np
import pandas as pd
import matplotlib.dates as mdates
import datetime as dt
import warnings # Turn off warnings
warnings.filterwarnings('ignore')

In [2]:
petition_df = pd.read_csv("~/data/sf-open-data/Petitions_to_the_Rent_Board.csv") # Import

In [3]:
# Rename some columns
petition_df = petition_df.rename(columns = {'Date Filed': 'Date',
                                             'Petition Source Zipcode': 'Zip Code',
                                             'Neighborhoods - Analysis Boundaries': 'Neighborhood Names',
                                             'Location': 'latlong',
                                             'Analysis Neighborhoods': 'Neighborhood Number',
                                             'Neighborhoods - Analysis Boundaries': 'Neighborhood Name'
                                            }
                                )

# Reorder columns, drop 'Priority', 'Appeal ID' and 'Prop I Petition'
petition_df = petition_df[['Date', 'Address', 'latlong', 'Zip Code',
    'Neighborhood Number', 'Neighborhood Name', 'SF Find Neighborhoods',
    'Current Police Districts', 'Current Supervisor Districts', 'Supervisor District',
                           
    'Petition ID',
       
    'Filing Party',
    'Ground (landlord): Capital Improvement',
    'Ground (landlord): Comparable Rents',
    'Ground (landlord): Extension of Time for Capital Improvements',
    'Ground (landlord): Intent to Withdraw (Ellis)',
    'Ground (landlord): Other',
    'Ground (landlord): Operating & Maintenance',
    'Ground (landlord): Substantial Rehabilitation',
    'Ground (landlord):  Costa-Hawkins',
    'Ground (landlord):  Exemption',
    'Ground (landlord): Rules and Regulations Section 1.21',
    'Ground (landlord): Rules and Regulations Section 6.14',
    'Ground (landlord): Rules and Regulations Section 6.15',
    'Ground (landlord): SRO Hotel Vistor Policy Petition',
    'Ground (landlord): Utility Passthrough Petition',
    'Ground (landlord): Owner Move-In Disability Determination',
    'Ground (landlord): Utility Passthrough Worksheet',
    'Ground (landlord): Non-Comparable Rents',

    'Ground (tenant): Decrease in Housing Services',
    'Ground (tenant): Failure to Repair and Maintain',
    'Ground (tenant): Passthrough Challenge',
    'Ground (tenant): Summary Petition',
    'Ground (tenant): Unlawful Rent Increase',
    'Ground (tenant): Wrongful Eviction Report',
    'Ground (tenant): Section 8 tenancy',
    'Ground (tenant): SRO Hotel Vistor Policy Petition',
    'Ground (tenant): Rules and Regulations Section 6.15',
    'Ground (tenant): Other',
    'Ground (tenant): Utility Passhtrough Hardship',
    'Ground (tenant): Water Revenue Bond Passthrough Hardship',
    'Ground (tenant): Capital Improvement Passthrough Hardship Petition',
    'Ground (tenant): Wrongful Severance of Housing Service',
    'Ground: Alternative Dispute Resolution', 
                           
    ':@computed_region_fyvs_ahh9', ':@computed_region_p5aj_wyqh',
    ':@computed_region_rxqg_mtj9', ':@computed_region_yftq_j783',
    ':@computed_region_bh8s_q3mv']]

# Clean up Zip Codes
petition_df['Zip Code'] = petition_df['Zip Code'].replace('[-][0-9]{4}', '', regex=True)
petition_df['Zip Code'] = petition_df['Zip Code'].str.extract('(\d+)', expand=False)
petition_df['Zip Code'] = petition_df['Zip Code'].apply(lambda x: np.int64(x) if not pd.isnull(x) else x)

# Standardizing some of the fields
petition_df['latlong'] = petition_df['latlong'].apply(lambda x: eval(x) if not pd.isnull(x) else x)
petition_df['Datetime'] = petition_df['Date'].apply(lambda x : dt.datetime.strptime(x, "%m/%d/%Y"))
petition_df['Timestamp'] = petition_df['Date'].apply(lambda x : mdates.datestr2num(x))

# Getting extra coordinates
petition_df['Latitude'] = petition_df['latlong'].apply(lambda x: x[0] if not pd.isnull(x) else x)
petition_df['Longitude'] = petition_df['latlong'].apply(lambda x: x[1] if not pd.isnull(x) else x)

# Add columns that include all the Petition IDs for mass petitions
petition_merge_df = petition_df.merge(
    petition_df.groupby(['Date', 'Address'])['Petition ID'] \
        .apply(list).to_frame('Petition ID List').reset_index(), on=['Date', 'Address'])

# Drop duplicates
petition_clean_df = petition_merge_df.drop(columns = ['Petition ID']) \
    .drop_duplicates(subset = ['Date', 'Address']).reset_index(drop = True)

# Add list length
petition_clean_df['Petition ID Len'] = petition_clean_df['Petition ID List'].map(len)

In [4]:
petition_clean_df.to_csv("~/data/sf-open-data/petitions-clean.csv", index=False) # Export
petition_clean_df

,Date,Address,latlong,Zip Code,Neighborhood Number,Neighborhood Name,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Supervisor District,...,:@computed_region_p5aj_wyqh,:@computed_region_rxqg_mtj9,:@computed_region_yftq_j783,:@computed_region_bh8s_q3mv,Datetime,Timestamp,Latitude,Longitude,Petition ID List,Petition ID Len
0,03/29/2019,100 Block Of Tiffany Avenue,"(37.74526077457311, -122.42122878627974)",94110.0,2.0,Bernal Heights,83.0,9.0,5.0,8.0,...,7.0,5.0,2.0,28859.0,2019-03-29,737147.0,37.745261,-122.421229,[T190501],1
1,03/29/2019,100 Block Of Laguna Street,"(37.77227352032319, -122.42534894614477)",94102.0,9.0,Hayes Valley,27.0,4.0,11.0,5.0,...,9.0,11.0,7.0,28852.0,2019-03-29,737147.0,37.772274,-122.425349,"[L190451, L190450]",2
2,03/29/2019,800 Block Of Greenwich Street,"(37.80189640951875, -122.4132706352494)",94133.0,32.0,Russian Hill,107.0,6.0,3.0,3.0,...,1.0,10.0,3.0,308.0,2019-03-29,737147.0,37.801896,-122.413271,[T190536],1
3,03/29/2019,2000 Block Of 45th Avenue,"(37.74835168769152, -122.50393869575339)",94116.0,35.0,Sunset/Parkside,39.0,10.0,7.0,4.0,...,8.0,3.0,1.0,29491.0,2019-03-29,737147.0,37.748352,-122.503939,[L190488],1
4,03/29/2019,300 Block Of Page Street,"(37.773615083644295, -122.42653977279896)",94102.0,9.0,Hayes Valley,26.0,4.0,11.0,5.0,...,9.0,11.0,7.0,28852.0,2019-03-29,737147.0,37.773615,-122.426540,[E190473],1
5,03/29/2019,100 Block Of Mason Street,"(37.78482477628598, -122.40933501549891)",94102.0,36.0,Tenderloin,19.0,5.0,3.0,3.0,...,10.0,10.0,14.0,28852.0,2019-03-29,737147.0,37.784825,-122.409335,[T190503],1
6,03/29/2019,1000 Block Of Union Street,"(37.7996251250944, -122.41613715345753)",94133.0,32.0,Russian Hill,107.0,6.0,6.0,2.0,...,1.0,1.0,5.0,308.0,2019-03-29,737147.0,37.799625,-122.416137,[T190499],1
7,03/29/2019,100 Block Of Ellert Street,"(37.73841125596938, -122.41719914654512)",94110.0,2.0,Bernal Heights,61.0,9.0,2.0,9.0,...,7.0,7.0,2.0,28859.0,2019-03-29,737147.0,37.738411,-122.417199,[L190478],1
8,03/29/2019,0 Block Of Imperial Avenue,NaN,94123.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2019-03-29,737147.0,NaN,NaN,[T190502],1
9,03/29/2019,0 Block Of Hill Street,"(37.75598162320782, -122.42212261001781)",94110.0,20.0,Mission,52.0,3.0,5.0,8.0,...,4.0,5.0,2.0,28859.0,2019-03-29,737147.0,37.755982,-122.422123,[L190435],1
